In [5]:
import requests
import mysql.connector
from bs4 import BeautifulSoup

def get_google_results(keyword, domain):
    
    results = []
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    url = f"https://www.google.com/search?q={keyword}&num=100&hl=en"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        for result in soup.select(".g"):
            link_element = result.select_one(".yuRUbf a")
            link = link_element["href"] if link_element else ""
            title_element = result.select_one(".yuRUbf a")
            title = title_element.text if title_element else ""
            description_element = result.select_one(".IsZvec")
            description = description_element.text if description_element else ""
            
            # Extract domain from the link
            domain_name = link.split("/")[2] if link else ""
            
            # Add the result to the list if it matches the target domain
            if domain in domain_name:
                results.append({"domain": domain_name, "link": link, "description": description})

    return results

# Connect to MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1080",
    database="project"
)

mycursor = mydb.cursor()
# # Create database 
# mycursor.execute("create database DomainRanker")

# # Create the table if it doesn't exist
# mycursor.execute("CREATE TABLE IF NOT EXISTS domain_name_availability ("
#                  "id INT AUTO_INCREMENT PRIMARY KEY,"
#                  "domain VARCHAR(255),"
#                  "link VARCHAR(255),"
#                  "description VARCHAR(255)"
#                  ")")

# Get the search term and domain name from user input
search_term = input("Enter a search keyword: ")
domain_name = input("Enter a domain name to check: ")

# Perform the Google search and get the results
results = get_google_results(search_term, domain_name)

# Insert the results into the table
for result in results:
    sql = "INSERT INTO domain_name_availability (domain, link, description) VALUES (%s, %s, %s)"
    val = (result["domain"], result["link"], result["description"])
    mycursor.execute(sql, val)

mydb.commit()

# Query the database to print the current rank of the domain being searched for
sql = f"SELECT id FROM domain_name_availability WHERE domain = '{domain_name}'"
mycursor.execute(sql)
result = mycursor.fetchone()
if result:
    print(f"The current rank of {domain_name} is {result[0]}")
else:
    print(f"{domain_name} is not currently ranked in the top 100 search results for {search_term}.")

# Query the database to print the entire table of search results
mycursor.execute("SELECT * FROM domain_name_availability")
result = mycursor.fetchall()
for row in result:
    print(row)

Enter a search keyword: Name search
Enter a domain name to check: Mohit
Mohit is not currently ranked in the top 100 search results for Name search.
(1, 'www.bluehost.in', 'https://www.bluehost.in/domains', '')
(2, 'orientwatch.in', 'https://orientwatch.in/', '')
(3, 'www.mynewcar.in', 'https://www.mynewcar.in/', '')
(4, 'www.car.info', 'https://www.car.info/en-se/', '')
(5, 'www.godaddy.com', 'https://www.godaddy.com/en-in/domains', '')
